### SGM: Sequence Generation Model for Multi-Label Classification    

#### 摘要

​     多标签分类在自然语言处理中还是一个重要挑战任务，远比单标签分类复杂，并且标签之间还相互关联。现存方法忽略标签的关联性。此外，文本不同部分对于预测不同标签贡献不同。已存在模型没有考虑这个问题。在本文，我们提出将多标签分类任务作为一个序列生成问题。应用一个带有新颖的解码结构的序列生成模型来解决它。拓展实验结果显示我们提出的模型超过之前工作。进一步分析实验结果证明已提出方法不仅获得标签的相关信息，而且在预测不同标签自动选择最大信息信息词。

#### 介绍

​    多标签分类是自然语言领域中一个重要任务，使用在很多真实世界场景中，例如文本分类，标签推荐，信息检索等等。

   在本文，受到序列到序列模型在机器翻译，抽象摘要，风格转换等领域的巨大成功激励，我们提出一个带有新颖解码的序列生成模型。提出的序列生成模型由基于注意力机制的解码器。解码器使用LSTM顺序生成标签，基于之前已预测标签来预测下一个标签。已提出模型考虑标签相关性，通过LSTM结构处理标签序列依赖。当模型用于预测不同标签注意力机制考虑文本不同部分作用。此外，带有全局嵌入的解码结构被提出通过整合整体信息来提升模型性能。

​    本文贡献点，如下：

​    我们提出将多标签分类任务作为一个序列生成问题，并且将标签关联关系加入考虑中。

   我们提出带有新颖解码结构的序列生成模型，当预测不同标签时不仅获得标签关联关系，而且自动选择最大信息字。

   拓展实验结果显示我们提出方法超过基线。进一步分析显示已提出方法在关联描述上是有效的。

#### 已提出方法

##### 综述

   首先，我们定义一些概念和描述多分类任务。给定标签空间有$L$个标签,$\mathcal{L}=\{l_1,l_2,...,l_L\}$,文本序列$x$包含m个字。从序列生成角度，多标签分类任务被建模为寻找一个最优标签序列${y^*}$,最大化条件概率分布$p{(y|x)}$:


$$
p(y|x) = \prod_{i=1}^np(y_i|y_1,y_2,...,y_{i-1},x)
$$
模型结构如下图1：

![1537587192595](https://github.com/jxz542189/paper_translate/raw/master/image/SGM1.png)

​        图1 已提出模型整体结构，MS表示masked softmax层，GE表示全局嵌入

​    首先，我们根据在训练集中标签的频率对每个样本的标签进行排序。高频率的放置在前面。此外，添加bos和eos符号添加到标签序列的头部和尾部。

   文本序列${x}$被编码为隐藏状态通过时间步注意力机制来整合为上下文向量。解码器将上下文向量$c_{t}$,解码器上一个隐藏状态$s_{t-1}$以及嵌入向量$g(y_{t-1})$作为输入来产生隐藏状态$s_t$，最后，掩饰softmax层用于概率分布$y_t$输出。

##### 序列生成

由基于注意力的编码器和解码器组成。

***Encoder***：$(\omega_1,\omega_2,...,\omega_m)$表示句子，有m个字，$\omega_i$是第i个字的one-hot表示。通过嵌入矩阵将$\omega_i$编码为稠密嵌入向量$x_i$。这里使用BiLSTM进行计算每个字的隐藏状态:
$$
\vec{h_i}=\vec{LSTM}(\vec{h_{i-1},x_i})
$$

$$
\overleftarrow{h_i}=\overleftarrow{LSTM}(\overleftarrow{h_{i+1}}, x_i)
$$

  获得第i个字最后隐藏描述：
$$
h_i=[\vec{h_i};\overleftarrow{h_i}]
$$
***注意力***：注意力机制产生一个上下文向量，聚焦在文本序列不同部分，整合这些信息字的隐藏描述，注意力机制赋予权值$\alpha_{ti}$给时间步t的第i个字：
$$
e_{ti}=v_{a}^Ttanh(W_as_t+U_ah_i)
$$

$$
\alpha_{ti}=\frac{exp(e_{ti})}{\sum_{j=1}^mexp(e_{tj})}
$$
最后上下文向量$c_t$将被传递时间步t的解码器：
$$
c_t=\sum_{i=1}^m\alpha_{ti}h_i
$$
***解码器***：在时间步t的解码器隐藏状态$s_t$如下计算：
$$
s_t=LSTM(s_{t-1},[g(y_{t-1});c_{t-1}])
$$
其中$g(y_{t-1})$表示基于分布$y_{t-1}$中概率最高的标签的嵌入。$y_{t-1}$如下计算：
$$
o_t=W_of(W_ds_t+V_dc_t)
$$

$$
y_t=softmax(o_t+I_t)
$$
$I_t$是一个掩饰向量用于阻止解码器预测重复标签：
$$
(I_t)_i=\left\{\frac{-\infty \quad 如果l_i已经在t-1时间步预测时}{0\quad 其他}\right\}
$$
训练阶段损失函数使用交叉损失函数。

##### 全局嵌入

前面预测模型是下一个标签预测基于之前标签，这导致之前预测错误会将继承，这称为爆炸偏置。为了解决爆炸偏置这个问题，本文提出一个新的解码结构，嵌入向量$g(y_{t-1})$在时间步t获得描述在时间步(t-1)所有整体信息。受到高速公路的适应门激励，这里本文介绍全局嵌入，定义e为分布$y_{t-1}$的最高概率的标签的嵌入。$\tilde{e}$在时间步t的平均权重嵌入，通过如下计算：
$$
\tilde{e}=\sum_{i=1}^Ly_{t-1}^{(i)}e_i
$$
其中$y_{t-1}^{(i)}$表示$y_{t-1}$的第i个元素。全局嵌入$g(y_{t-1})$通过如下获得:
$$
g(y_{t-1})=(1-H)\odot e + H\odot \tilde{e}
$$

$$
H=W_1e+W_2\tilde{e}
$$

全局嵌入$g(y_{t-1})$是个优化整合源嵌入和权值平均嵌入。$y_{t-1}$包含所有可能信息。通过考虑每个标签，可以减少由于之前时间步预测错误的标签。

#### 实验部分

使用指标：汉明损失，$Micro-F_i$



### SGM代码解析

```python
class global_attention(nn.Module):
    def __init__(self, hidden_size, activation=None):
        super(global_attention, self).__init__()
        self.linear_in = nn.Linear(hidden_size, hidden_size)
        self.linear_out = nn.Linear(2*hidden_size, hidden_size)
        self.softmax = nn.Softmax()
        self.tanh = nn.Tanh()
        self.activation = activation

    def forward(self, x, context):
        gamma_h = self.linear_in(x).unsqueeze(2)    # batch * size * 1
        if self.activation == 'tanh':
            gamma_h = self.tanh(gamma_h)
        weights = torch.bmm(context, gamma_h).squeeze(2)   # batch * time
        weights = self.softmax(weights)   # batch * time
        c_t = torch.bmm(weights.unsqueeze(1), context).squeeze(1) # batch * size
        output = self.tanh(self.linear_out(torch.cat([c_t, x], 1)))
        return output, weights
'''
全局注意力：
首先通过全连接，接着与上下文相乘获得权重，此时更新上下文向量，整合上下文向量和原始向量，最后通过全连接
'''
```

```python
'''多层LSTMCell'''
class StackedLSTM(nn.Module):
    def __init__(self, num_layers, input_size, hidden_size, dropout):
        '''
        num_layers:LSTMCell单元层数
        input_size:输入单元数
        hidden_size:隐藏单元数
        dropout:
        '''
        super(StackedLSTM, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.num_layers = num_layers
        self.layers = nn.ModuleList()

        for i in range(num_layers):
            self.layers.append(nn.LSTMCell(input_size, hidden_size))
            input_size = hidden_size
'''每一时间步的隐藏层状态和细胞输出分别通过堆叠多层结果得到'''
    def forward(self, input, hidden):
        h_0, c_0 = hidden
        h_1, c_1 = [], []
        for i, layer in enumerate(self.layers):
            h_1_i, c_1_i = layer(input, (h_0[i], c_0[i]))
            input = h_1_i
            if i + 1 != self.num_layers:
                input = self.dropout(input)
            h_1 += [h_1_i]
            c_1 += [c_1_i]

        h_1 = torch.stack(h_1)
        c_1 = torch.stack(c_1)
        return input, (h_1, c_1)
```

```python
'''
lstm对句子进行编码
'''
class rnn_encoder(nn.Module):

    def __init__(self, config, vocab_size, embedding=None):
        super(rnn_encoder, self).__init__()
        if embedding is not None:
            self.embedding = embedding
        else:
            self.embedding = nn.Embedding(vocab_size, config.emb_size)
        self.rnn = nn.LSTM(input_size=config.emb_size, hidden_size=config.encoder_hidden_size,
                           num_layers=config.num_layers, dropout=config.dropout, bidirectional=config.bidirec)
        self.config = config

    def forward(self, input, lengths):
        embs = pack(self.embedding(input), lengths)
        outputs, (h, c) = self.rnn(embs)
        outputs = unpack(outputs)[0]
        '''根据是否是双向'''
        if not self.config.bidirec:
            return outputs, (h, c)
        else:
            batch_size = h.size(1)
            h = h.transpose(0, 1).contiguous().view(batch_size, -1, 2 * self.config.encoder_hidden_size)
            c = c.transpose(0, 1).contiguous().view(batch_size, -1, 2 * self.config.encoder_hidden_size)
            state = (h.transpose(0, 1), c.transpose(0, 1))
            return outputs, state
```

```python
class gated_rnn_encoder(nn.Module):
    def __init__(self, config, vocab_size, embedding=None):
        super(gated_rnn_encoder, self).__init__()
        if embedding is not None:
            self.embedding = embedding
        else:
            self.embedding = nn.Embedding(vocab_size, config.emb_size)
        self.rnn = nn.LSTM(input_size=config.emb_size, hidden_size=config.encoder_hidden_size,
                          num_layers=config.num_layers, dropout=config.dropout)
        self.gated = nn.Sequential(nn.Linear(config.encoder_hidden_size, 1), nn.Sigmoid())
    
    def forward(self, input, lengths):
        embs = pack(self.embedding(input), lengths)
        outputs, state = self.rnn(embs)
        outputs = unpack(outputs)[0]
        p = self.gated(outputs)
        outputs = outputs * p
        return outputs, state
```

```python
'''
rnn解码器

'''
class rnn_decoder(nn.Module):

    def __init__(self, config, vocab_size, embedding=None, score_fn=None):
        super(rnn_decoder, self).__init__()
        if embedding is not None:
            self.embedding = embedding
        else:
            self.embedding = nn.Embedding(vocab_size, config.emb_size)
        self.rnn = StackedLSTM(input_size=config.emb_size, hidden_size=config.decoder_hidden_size,
                           num_layers=config.num_layers, dropout=config.dropout)

        self.score_fn = score_fn
        if self.score_fn.startswith('general'):
            self.linear = nn.Linear(config.decoder_hidden_size, config.emb_size)
        elif score_fn.startswith('concat'):
            self.linear_query = nn.Linear(config.decoder_hidden_size, config.decoder_hidden_size)
            self.linear_weight = nn.Linear(config.emb_size, config.decoder_hidden_size)
            self.linear_v = nn.Linear(config.decoder_hidden_size, 1)
        elif not self.score_fn.startswith('dot'):
            self.linear = nn.Linear(config.decoder_hidden_size, vocab_size)

        if hasattr(config, 'att_act'):
            activation = config.att_act
            print('use attention activation %s' % activation)
        else:
            activation = None

        self.attention = models.global_attention(config.decoder_hidden_size, activation)
        self.hidden_size = config.decoder_hidden_size
        self.dropout = nn.Dropout(config.dropout)
        self.config = config

        if self.config.global_emb:
            self.gated1 = nn.Linear(config.emb_size, config.emb_size)
            self.gated2 = nn.Linear(config.emb_size, config.emb_size)

    def forward(self, inputs, init_state, contexts):
        if not self.config.global_emb:
            embs = self.embedding(inputs)
            outputs, state, attns = [], init_state, []
            for emb in embs.split(1):
                output, state = self.rnn(emb.squeeze(0), state)
                output, attn_weights = self.attention(output, contexts)
                output = self.dropout(output)
                outputs += [output]
                attns += [attn_weights]
            outputs = torch.stack(outputs)
            attns = torch.stack(attns)
            return outputs, state
        else:
            outputs, state, attns = [], init_state, []
            embs = self.embedding(inputs).split(1)
            max_time_step = len(embs)
            emb = embs[0]
            output, state = self.rnn(emb.squeeze(0), state)
            output, attn_weights = self.attention(output, contexts)
            output = self.dropout(output)
            soft_score = F.softmax(self.linear(output))
            outputs += [output]
            attns += [attn_weights]

            batch_size = soft_score.size(0)
            a, b = self.embedding.weight.size()

            for i in range(max_time_step-1):
                emb1 = torch.bmm(soft_score.unsqueeze(1), self.embedding.weight.expand((batch_size, a, b)))
                emb2 = embs[i+1]
                gamma = F.sigmoid(self.gated1(emb1.squeeze())+self.gated2(emb2.squeeze()))
                emb = gamma * emb1.squeeze() + (1 - gamma) * emb2.squeeze()
                output, state = self.rnn(emb, state)
                output, attn_weights = self.attention(output, contexts)
                output = self.dropout(output)
                soft_score = F.softmax(self.linear(output))
                outputs += [output]
                attns += [attn_weights]
            outputs = torch.stack(outputs)
            attns = torch.stack(attns)
            return outputs, state

    def compute_score(self, hiddens):
        if self.score_fn.startswith('general'):
            if self.score_fn.endswith('not'):
                scores = torch.matmul(self.linear(hiddens), Variable(self.embedding.weight.t().data))
            else:
                scores = torch.matmul(self.linear(hiddens), self.embedding.weight.t())
        elif self.score_fn.startswith('concat'):
            if self.score_fn.endswith('not'):
                scores = self.linear_v(torch.tanh(self.linear_query(hiddens).unsqueeze(1) + \
                                      self.linear_weight(Variable(self.embedding.weight.data)).unsqueeze(0))).squeeze(2)
            else:
                scores = self.linear_v(torch.tanh(self.linear_query(hiddens).unsqueeze(1) + \
                                      self.linear_weight(self.embedding.weight).unsqueeze(0))).squeeze(2)
        elif self.score_fn.startswith('dot'):
            if self.score_fn.endswith('not'):
                scores = torch.matmul(hiddens, Variable(self.embedding.weight.t().data))
            else:
                scores = torch.matmul(hiddens, self.embedding.weight.t())
        else:
            scores = self.linear(hiddens)
        return scores

    def sample(self, input, init_state, contexts):
        inputs, outputs, sample_ids, state = [], [], [], init_state
        attns = []
        inputs += input
        max_time_step = self.config.max_tgt_len
        soft_score = None
        mask = None
        for i in range(max_time_step):
            output, state, attn_weights = self.sample_one(inputs[i], soft_score, state, contexts, mask)
            if self.config.global_emb:
                soft_score = F.softmax(output)
            predicted = output.max(1)[1]
            inputs += [predicted]
            sample_ids += [predicted]
            outputs += [output]
            attns += [attn_weights]
            if self.config.mask:
                if mask is None:
                    mask = predicted.unsqueeze(1).long()
                else:
                    mask = torch.cat((mask, predicted.unsqueeze(1)), 1)

        sample_ids = torch.stack(sample_ids)
        attns = torch.stack(attns)
        return sample_ids, (outputs, attns)

    def sample_one(self, input, soft_score, state, contexts, mask):
        if self.config.global_emb:
            batch_size = contexts.size(0)
            a, b = self.embedding.weight.size()
            if soft_score is None:
                emb = self.embedding(input)
            else:
                emb1 = torch.bmm(soft_score.unsqueeze(1), self.embedding.weight.expand((batch_size, a, b)))
                emb2 = self.embedding(input)
                gamma = F.sigmoid(self.gated1(emb1.squeeze())+self.gated2(emb2.squeeze()))
                emb = gamma * emb1.squeeze() + (1 - gamma) * emb2.squeeze()
        else:
            emb = self.embedding(input)
        output, state = self.rnn(emb, state)
        hidden, attn_weigths = self.attention(output, contexts)
        output = self.compute_score(hidden)
        if self.config.mask:
            if mask is not None:
                output = output.scatter_(1, mask, -9999999999)
        return output, state, attn_weigths
```